# Разработка алгоритма для защиты данных

Задача - методом преобразования данных защитить личную информацию клиентов страховой компании. Важное условие - сделать это надо так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

### План решения задачи


##### Часть 1. Подготовка данных:
* [1. Загрузка и изучение данных](#1-bullet)
* [2. Разбиение данных на train и test](#2-bullet)

##### Часть 2. Теоретическое обоснование и практическое применение:
* [1. Умножение матриц](#3-bullet)
* [2. Алгоритм преобразования](#4-bullet)
* [3. Проверка алгоритма](#5-bullet)

## Часть 1. Подготовка данных 

### 1. Загрузка и изучение данных<a id='1-bullet'></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


### 2. Разбиение данных на train и test <a id='2-bullet'></a>

In [5]:
X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']

X.shape, y.shape

((5000, 4), (5000,))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=124)

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3750, 4), (1250, 4), (3750,), (1250,))

## Часть 2. Теоретическое обоснование и практическое применение

### 1. Умножение матриц <a id='3-bullet'></a>

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится

**Обоснование:** 
Предсказания до кодировки

$
a = Xw
$ , где $ w = (X^T X)^{-1} X^T y $

Предсказания после кодировки

$
a_p = XPw
$ , где $ w = ((XP)^TXP)^{-1}(XP)^Ty $

Для доказательства необходимо подставить $w_p$ и раскрыть скобки

$ a_p = XP((XP)^T XP)^{-1}(XP)^T y = XP(P^TX^T XP)^{-1} P^TX^T y = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty = Xw $

Получаем соответствие параметров исходной и преобразованной задачи

### 2. Алгоритм преобразования <a id='4-bullet'></a>

Нам нужно защитить данные своих клиентов. Поступим следующим образом:
- Создаем случайную квадратную матрицу для того, чтобы мы могли умножить ее на матрицу признаков. Такое преобразование сделает исходные значения признаков невосстановимыми.
- Проверяем ее на обратимость, так как у нас должна быть возможность расшифровать нашу зашифрованную матрицу
- Перемножаем матрицу признаков и случайную
- Применяем линейную регрессию для двух датасетов: изначального и измененного, и сравниваем качество с помощью метрики R2

В предыдущем пункте мы выяснили, что умножение матрицы признаков (X) на случайную матрицу (P) не влияет на качество линейной регрессии. Пришло время проверить это на практике.

### 3. Проверка алгоритма <a id='5-bullet'></a>

Домножим на одну случайную матрицу и train и test

In [8]:
state = np.random.RandomState(240)

random_matrix = state.normal(size=(X_train.shape[1], X_train.shape[1]))

X_train_new = X_train.dot(random_matrix)
X_test_new = X_test.dot(random_matrix)

In [9]:
random_matrix.shape, X_train.shape, X_train_new.shape, X_test.shape, X_test_new.shape

((4, 4), (3750, 4), (3750, 4), (1250, 4), (1250, 4))

In [10]:
# проверим на обратимость

random_matrix_inv = np.linalg.inv(random_matrix)

print(random_matrix)

[[-0.58232733 -1.72710963 -0.69454013  0.76951838]
 [ 1.0872238  -0.62389009 -0.19859188  1.12158004]
 [ 0.29351644  0.29375841  0.5833624   0.36534502]
 [ 1.58036064  0.92187528 -0.95970967  0.11480781]]


Обратимость случайной матрицы важна потому, что мы пользовались ей при доказательстве (при раскрытии скобки $((XP)^T XP)^{-1}$).
</div>

Обучаем две линейный регрессии: одну на изначальных данных, вторую на преобразованных. Далее проверяем качество с помощью метрики R2 и сравниваем результаты

In [12]:
lin_reg = LinearRegression()

# Изначальные данные

lin_reg.fit(X_train, y_train)
predict_first = lin_reg.predict(X_test)
print(r2_score(y_test, predict_first))

0.44371454562237145


In [13]:
# Измененные данные

lin_reg.fit(X_train_new, y_train)
predict_second = lin_reg.predict(X_test_new)
print(r2_score(y_test, predict_second))

0.44371454562272106


Качество предсказаний отличается незначительно. Проверка алгоритма на практике позволила подтвердить наше теоретическое предположение о том, что в случае умножения признаков на обратимую матрицу качество линейной регрессии не изменится. 